### Imports

In [1]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.circuit.library import ZGate
from qiskit_aer import AerSimulator

from qiskit.circuit import QuantumRegister
from qiskit.dagcircuit import DAGCircuit
from qiskit_aer.library.save_instructions import SaveStatevector
from qiskit.visualization.dag_visualization import dag_drawer

from LogicalQ.Experiments import execute_circuits
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code

qc = QuantumCircuit(1)
#qc.barrier([0,1], label="insert_before_this")
#qc.ry(0.5, 1)
qc.t(0)

simulator = AerSimulator()


print("Original Circuit:")
print(qc)

Original Circuit:
   ┌───┐
q: ┤ T ├
   └───┘


In [2]:
lg_qc = LogicalCircuit.from_physical_circuit(qc, **steane_code)
lg_qc.encode(0, max_iterations=1, initial_states=[1])
lg_qc.measure_all()

### Original implementation of inefficient algorithm

In [3]:
def insert_save_statevector(lg_qc):
    """
    Traverses an original DAG, inserts a SaveStatevector instruction before
    a specific 'box' node, and returns a new DAG.

    Args:
        original_dag (DAGCircuit): The DAG to be traversed.

    Returns:
        DAGCircuit: The new DAG with the instruction inserted.
    """
    
    original_dag = circuit_to_dag(lg_qc)
    
    new_dag = DAGCircuit()
    for qreg in original_dag.qregs.values():
        new_dag.add_qreg(qreg)
    for creg in original_dag.cregs.values():
        new_dag.add_creg(creg)

    insertion_complete = False

    for node in original_dag.topological_op_nodes():
        op_name = getattr(node.op, 'name', None)
        op_label = getattr(node.op, 'label', None)

        #print(f"Processing node: {op_name}, with label: {op_label}")

        if (not insertion_complete and
            op_name == "box" and
            op_label and op_label.split(":")[0] == "logical.qec.measure"):
            
            print("Condition met! Inserting SaveStatevector before the 'box'.")
            
            qubits = []
            regs = list(new_dag.qregs.values())
            for reg in regs:
                qubits = qubits + [qubit for qubit in reg]
            print(qubits)
            save_inst = SaveStatevector(len(qubits), "statevector")
            new_dag.apply_operation_back(save_inst, qubits)

            insertion_complete = True
        
        new_dag.apply_operation_back(
            node.op,
            qargs=node.qargs,
            cargs=node.cargs
        )
    
    new_lg_qc = dag_to_circuit(new_dag)
    
    return new_dag, new_lg_qc

In [4]:
new_dag, new_lg_qc = insert_save_statevector(lg_qc)
#new_lg_qc.draw(output='mpl')

Condition met! Inserting SaveStatevector before the 'box'.
[<Qubit register=(7, "qlog0"), index=0>, <Qubit register=(7, "qlog0"), index=1>, <Qubit register=(7, "qlog0"), index=2>, <Qubit register=(7, "qlog0"), index=3>, <Qubit register=(7, "qlog0"), index=4>, <Qubit register=(7, "qlog0"), index=5>, <Qubit register=(7, "qlog0"), index=6>, <AncillaQubit register=(3, "qanc0"), index=0>, <AncillaQubit register=(3, "qanc0"), index=1>, <AncillaQubit register=(3, "qanc0"), index=2>, <AncillaQubit register=(2, "qlogical_op0"), index=0>, <AncillaQubit register=(2, "qlogical_op0"), index=1>, <Qubit register=(2, "qsetter"), index=0>, <Qubit register=(2, "qsetter"), index=1>]


In [ ]:
new_lg_qc.draw(output='mpl')

In [5]:
img = dag_drawer(new_dag)
img.save(r"C:\Users\nolan\Downloads\dag.png")

In [ ]:
results = execute_circuits(
    new_lg_qc,
    backend=simulator,
    shots=1,
    memory=True
    )

In [7]:
statevector = results[0].get_statevector()

In [8]:
statevector.draw(output='latex')

<IPython.core.display.Latex object>

### Development of more efficient algorithm

In [ ]:
from qiskit.circuit import QuantumRegister
from qiskit.dagcircuit import DAGCircuit
from qiskit_aer.library.save_instructions import SaveStatevector

dag = circuit_to_dag(lg_qc)

target_node = None
for node in dag.op_nodes():
    #if getattr(node.op, 'label', None) == 'insert_before_this':
    #    target_node = node
    #    break
    name, label = getattr(node.op, 'name', None), getattr(node.op, 'label', None)
    print(f"{name}, {label}")
    if name == "box":
        #print(instruction.operation.label)
        if label.split(":")[0] == "logical.qec.measure":
            target_node = node

if target_node:
    #print("found!")
    predecessor_nodes = dag.predecessors(target_node)
    num_predecessors = sum(1 for e in predecessor_nodes)
    print(num_predecessors)
    reg = QuantumRegister(num_predecessors)
    reg_ids = [reg[i] for i in range(len(reg))]
    print(len(reg_ids))
    #mini_dag = DAGCircuit()
    #mini_dag.add_qreg(reg)
    #mini_dag.apply_operation_back(SaveStatevector(num_predecessors), qargs=reg_ids)
    #dag.substitute_node_with_dag(target_node, mini_dag, wires=reg_ids)
    #new_qc = dag_to_circuit(dag)
    #print("\nModified Circuit:")
    
    sub_dag_circuit = target_node.op.definition
    sub_dag = circuit_to_dag(sub_dag_circuit)
    qubits = node.qargs
    save_inst = SaveStatevector(len(qubits), "statevector")
    sub_dag.apply_operation_front(save_inst, qargs=qubits)
    dag.substitute_node_with_dag(target_node, sub_dag, wires=reg_ids)

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit

n_cbits = 2
n_qbits = 2

creg = ClassicalRegister(n_cbits)
qreg = QuantumRegister(n_qbits)

qc = QuantumCircuit(qreg, creg)

qc = QuantumCircuit(qreg, creg)
qc.h(0)
qc.cx(0, 1)
qc.barrier()

qc.draw("mpl")

In [ ]:
lg_qc.draw()

In [ ]:
print(lg_qc.data)

### Test using insertion using in-built with execute_circuits

In [3]:
results = execute_circuits(
    lg_qc,
    backend=simulator,
    shots=1,
    memory=True,
    save_statevector=True
    )

i got here


In [4]:
statevector = results[0].get_statevector()

In [5]:
statevector.draw(output='latex')

<IPython.core.display.Latex object>

In [6]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
for item in L:
    item = 3
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]